In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2 
import numpy as np
import h5py
from tensorflow.keras.models import load_model

In [2]:
cnn_model = load_model('ASL_MNIST_CNN_20.h5')

In [37]:
lookUpTable1 = np.empty((1,256), np.uint8)
for i in range(256):
    lookUpTable1[0,i] = np.clip(pow(i / 255.0, 0.5) * 255.0, 0, 255)
        
lookUpTable2 = np.empty((1,256), np.uint8)
for i in range(256):
    lookUpTable2[0,i] = np.clip(pow(i / 255.0, 16) * 255.0, 0, 255)

In [43]:
### Setting the number of the camera
cap = cv2.VideoCapture(0)
### Setting the resolusion of the camera (720p)
img_width = 1280
img_height = 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, img_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, img_height)

cut_point1 = int((img_width - img_height ) / 2) - 1
cut_point2 = int((img_width + img_height ) / 2)

org1 = int(cut_point1 / 2)
org2 = int(img_height / 2)

org = (int(cut_point1 / 2), int(img_height / 2))
fontFace = cv2.FONT_HERSHEY_COMPLEX
fontScale = 5
fontcolor = (0, 0, 255) # BGR
thickness = 5 
lineType = 4

while 1:
    ret, frame_o = cap.read()
    
    
    ####################################################
    # Frame processing
    
    frame = frame_o[:, cut_point1:cut_point2]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    frame = cv2.LUT(frame, lookUpTable1)
    
    frame = cv2.GaussianBlur(frame, (31,31),0)
    ret2, frame_mask1 = cv2.threshold(frame ,0 ,255 , cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    frame = cv2.LUT(frame, lookUpTable2)

#     frame_mask2 = cv2.adaptiveThreshold(frame,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,301,2)

    
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
#     closed = cv2.morphologyEx(frame_mask1, cv2.MORPH_CLOSE, kernel)


    
    frame_mask = frame_mask1==255
    frame = 255 - (255-frame) * frame_mask
    
    frame = cv2.resize(frame,(28,28))
    ####################################################
    
    ####################################################
    img = frame.reshape(-1,28,28,1)
    y_pred = cnn_model.predict(img)
    num = np.argmax(y_pred)   
    
    if num <= 8:
        letter = chr(num + 65)
    else:
        letter = chr(num + 66)

    cv2.putText(frame_o, letter, org, fontFace, fontScale, fontcolor, thickness, lineType)
    cv2.rectangle(frame_o, (cut_point1 ,0), (cut_point2, img_height-1), (0, 255,0),5)
    cv2.imshow('video',frame_o)
    cv2.imshow('v2',img[0])
    
    key = cv2.waitKey(1)
    ### Press Esc to quit detecting
    if key == 27:
        break
        
        

cap.release()
cv2.destroyAllWindows()        